In [168]:
import glob
import time
import datetime
import subprocess
from subprocess import Popen, PIPE

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)

## Import historical game by game data from FiveThirtyEight GitHub Repo

[FiveThirtyEight Data Repo](https://github.com/fivethirtyeight/data)

In [90]:
url = "https://projects.fivethirtyeight.com/nba-model/nba_elo.csv"
df_elo_1946_2018 = pd.read_csv(url).astype({'date': 'datetime64[ns]'})
df_elo_1946_2018.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carmelo1_pre,carmelo2_pre,carmelo1_post,carmelo2_post,carmelo_prob1,carmelo_prob2,score1,score2
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.0,1300.0000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0
1,1946-11-02,1947,0,NaN,CHS,NYK,1300.0,1306.7233,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0
2,1946-11-02,1947,0,NaN,PRO,BOS,1300.0,1300.0000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0
3,1946-11-02,1947,0,NaN,STB,PIT,1300.0,1300.0000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0
4,1946-11-02,1947,0,NaN,DTF,WSC,1300.0,1300.0000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0


### Analyze fields

See [FiveThirtyEight NBA Predictions](https://projects.fivethirtyeight.com/2018-nba-predictions/) for explanation of Elo and CARM-Elo

| Name           |  Type         | Description                                                                        
| -------------  |-------------  | ----------------------------------------------------------------------------------|
| date           | datetime      |   Game date                                                                       |                      
| season         | int           |   End year of season                                                              |
| neutral        | bool          |   1=neutral home court                                                            |
| playoff        | string        |   1946-2015: t = playoffs. Starting in 2016: q=conference quarterfinals, s=conference semifinals, c=conference final,f=finals                                                                 |
| team1          | string        |   Home team initials                                                              |
| team2          | string        |   Away team initials                                                              |
| elo1_pre       | double        |   Home team Elo before game                                                       |
| elo2_pre       | double        |   Away team Elo before game                                                       |
| elo_prob1      | double        |   Win probability based on elo1_pre                                               |
| elo_prob2      | double        |   Win probability based on elo2_pre                                               |
| elo1_post      | double        |   Home team Elo after game                                                        |
| elo2_post      | double        |   Away team Elo after game                                                        |
| carmelo1_pre\* | double        |   Home team CARM-Elo before game                                                  |
| carmelo2_pre\* | double        |   Away team CARM-Elo before game                                                  |
| carmelo1_post\*| double        |   Home team CARM-Elo after game                                                   |
| carmelo2_post\*| double        |   Away team CARM-Elo after game                                                   |
| carmelo_prob1\*| double        |   Win probability based on carmelo1_pre                                           |
| carmelo_prob2\*| double        |   Win probability based on carmelo2_pre                                           |
| score1         | int           |   Home team final score                                                           |
| score2         | int           |   Away team final score                                                           |

\* data only available from 2015-2018

#### Determine CARM-Elo starting point

In [8]:
df_carm_elo = df[["date", "season", "team1", "team2", "carmelo1_pre", "carmelo2_pre", "carmelo1_post", "carmelo2_post"]]
df_carm_elo[~(pd.isnull(df_carm_elo["carmelo1_pre"]))].sort_values("date").head()

,date,season,team1,team2,carmelo1_pre,carmelo2_pre,carmelo1_post,carmelo2_post
63157,2015-10-27,2016,ATL,DET,1542.664875,1451.827385,1521.809995,1472.682265
63158,2015-10-27,2016,CHI,CLE,1564.372491,1732.025482,1570.473937,1725.924036
63159,2015-10-27,2016,GSW,NOP,1730.513765,1555.126845,1734.342589,1551.298021
63173,2015-10-28,2016,LAL,MIN,1317.548331,1345.379348,1312.320723,1350.606956
63172,2015-10-28,2016,SAC,LAC,1487.260869,1671.590488,1481.391159,1677.460198


#### CARM-Elo data available starting with 2015-2016 season

## Import historical season data from basketball-reference

[Basketball-Reference Miscellaneous season stats](https://www.basketball-reference.com/leagues/NBA_2018.html#misc_stats::none)

** NOTE: ORtg and DRtg available starting from 1950-1951 NBA season **

In [204]:
def get_season_data(end_year):
    ''' 
    get cumulative statistics for season specified by end_year
    
    end_year: int, year to query (ex: 2018 queries 2017-2018 season)
    
    returns Pandas dataframe w/ basketball-reference.com's miscellaneous stats table for season specified by end_year
    '''
    from bs4 import Comment
    html = "https://www.basketball-reference.com/leagues/NBA_{}.html".format(end_year)
    result = requests.get(html)
    soup = BeautifulSoup(result.content, "html.parser")
    # html tree is strange...table is wrapped inside a comment
    table = [c for c in (soup.find('div', id="all_misc_stats")).children if type(c) == Comment][0]
    # parse table with pandas
    df = pd.read_html(table, header=1)[0]
    df["Season"] = end_year
    return df

### Option 1) Download latest data

In [209]:
def generate_season_summaries_for(start_year, end_year):
    '''
    generate dataframe with basketball-reference season summary data (see https://www.basketball-reference.com/leagues/NBA_2018.html#misc_stats::none) for all seasons [start_year, end_year] (inclusive)

    start_year: int, lower bound for year (ex: 2000 = 1999-2000 NBA season)
    end_year: int, upper bound for year

    returns dataframe with season summary data for all seasons [start_year, end_year] (inclusive)
    '''
    assert start_year >= 1950, "Start year must be 1950 or later"
    df_season_summaries = pd.concat([get_season_data(i) for i in range(start_year, end_year+1)])
    # Playoff teams labeled with * in basketball-reference data. Remove distinction for easier grouping.
    df_season_summaries["Team"] =  df_season_summaries["Team"].map(lambda s: s.replace("*", ""))
    df_season_summaries["NetRtg"] = df_season_summaries["ORtg"] - df_season_summaries["DRtg"]
    df_season_summaries = df_season_summaries.set_index(["Season", "Team"])
    # Re-order columns
    cols = df_season_summaries_1951_2018.columns.values.tolist()
    df_season_summaries = df_season_summaries_1951_2018.reindex_axis(cols[:cols.index("Pace")] + ["NetRtg"] + cols[cols.index("Pace"):-1], axis=1)
    return df_season_summaries

In [210]:
# ORtg and DRtg available starting from 1950-51 season 
start_year = 1951
end_year = 2018
generate_season_summaries_for(start_year, end_year).to_csv("../Data/nba_season_summaries_{}_{}.csv".format(start_year, end_year))

### Option 2) Use saved offline file

In [5]:
start_year = 1951
end_year = 2018
df_season_summaries_1951_2018 = pd.read_csv("../Data/nba_season_summaries_{}_{}.csv".format(start_year, end_year), index_col=[0, 1])
# For MultiIndex slicing support
df_season_summaries_1951_2018.sort_index(inplace=True) 
df_season_summaries_1951_2018.head()

Rk  Age     W     L    PW    PL   MOV   SOS  \
Season Team                                                                   
1951   Baltimore Bullets       8.0  NaN  24.0  42.0  27.0  39.0 -2.29  0.35   
       Boston Celtics          6.0  NaN  39.0  30.0  34.0  35.0 -0.26 -0.15   
       Fort Wayne Pistons      7.0  NaN  32.0  36.0  29.0  39.0 -1.87  0.07   
       Indianapolis Olympians  9.0  NaN  31.0  37.0  27.0  41.0 -2.37  0.37   
       League Average          NaN  NaN   NaN   NaN  32.0  32.0  0.00  0.00   

                                SRS  ORtg  DRtg  NetRtg   Pace    FTr  3PAr  \
Season Team                                                                   
1951   Baltimore Bullets      -1.94  83.9  86.3    -2.4   96.8  0.364   NaN   
       Boston Celtics         -0.41  87.3  87.6    -0.3   95.9  0.431   NaN   
       Fort Wayne Pistons     -1.81  82.0  83.8    -1.8  102.4  0.403   NaN   
       Indianapolis Olympians -2.00  83.6  86.0    -2.4   96.4  0.329   NaN   
       League Average          0.00  85.1  85.1     0.0   97.4  0.399   NaN   

                                 TS%   eFG%  TOV%  ORB%  FT/FGA  eFG%.1  \
Season Team                                                               
1951   Baltimore Bullets       0.421  0.353   NaN   NaN   0.271     NaN   
       Boston Celtics          0.441  0.368   NaN   NaN   0.312     NaN   
       Fort Wayne Pistons      0.410  0.338   NaN   NaN   0.290     NaN   
       Indianapolis Olympians  0.420  0.363   NaN   NaN   0.236     NaN   
       League Average          0.428  0.357   NaN   NaN   0.293     NaN   

                               TOV%.1  DRB%  FT/FGA.1  \
Season Team                                             
1951   Baltimore Bullets          NaN   NaN       NaN   
       Boston Celtics             NaN   NaN       NaN   
       Fort Wayne Pistons         NaN   NaN       NaN   
       Indianapolis Olympians     NaN   NaN       NaN   
       League Average             NaN   NaN       NaN   

                                                    Arena   Attend.  Attend./G  
Season Team                                                                     
1951   Baltimore Bullets               Baltimore Coliseum       NaN        NaN  
       Boston Celtics                       Boston Garden  197888.0        NaN  
       Fort Wayne Pistons      North Side High School Gym       NaN        NaN  
       Indianapolis Olympians           Hinkle Fieldhouse       NaN        NaN  
       League Average                                 NaN  197888.0        NaN

### Analyze fields

| Name      |  Type  | Description                                                                                   |
| --------- |--------| --------------------------------------------------------------------------------------------- |
| Rk        | double |   Rank (used to index for sorting within webpage)                                             |
| Age       | double |   Age of Player at the start of February 1st of that season.                                  |
| W         | double |   Wins                                                                                        |
| L         | double |   Losses                                                                                      |
| PW        | double |   Pythagorean wins, i.e., expected wins based on points scored and allowed                    |
| PL        | double |   Pythagorean losses, i.e., expected losses based on points scored and allowed                |
| MOV       | double |   Margin of Victory                                                                           |
| SOS       | double |   Strength of Schedule; a rating of strength of schedule. The rating is denominated in points above/below average, where zero is average.                                                                          |
| SRS       | double |   Simple Rating System; a team rating that takes into account average point differential and strength of schedule. The rating is denominated in points above/below average, where zero is average.                |
| ORtg      | double |   An estimate of points produced (players) or scored (teams) per 100 possessions              |  
| DRtg      | double |   An estimate of points allowed per 100 possessions                                           |
| NetRtg    | double |   ORtg - DRtg                                                                                 |
| Pace      | double |   An estimate of possessions per 48 minutes                                                   |
| FTr       | double |   Number of FT Attempts Per FG Attempt                                                        |
| 3PAr\*\*  | double |   Percentage of FG Attempts from 3-Point Range                                                |
| TS%       | double |   A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.                                                                                              |
| eFG%      | double |   Adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.                                                                                                                |
| TOV%\*    | double |   An estimate of turnovers committed per 100 plays.                                           |
| ORB%\*    | int    |   An estimate of the percentage of available offensive rebounds a team grabbed.               |
| FT/FGA    | int    |   Free Throws Per Field Goal Attempt                                                          |
| eFG%.1    | double |    Opponent Effective Field Goal Percentage                                                   |
| TOV%.1\*  | double |   Opponent Turnover Percentage                                                                |
| DRB%\*    | int    |   An estimate of the percentage of available defensive rebounds a team grabbed.               |
| FT/FGA.1\*| int    |   Opponent Free Throws Per Field Goal Attempt                                                 |    
| Arena     | string |   Home Arena                                                                                  |
| Attend.   | double |   Cumulative home attendance                                                                  |
| Attend./G | double |   Attendance per home game at the team's primary arena                                        |

\* data only available from 1974-2018

\*\* data only available from 1980-2018

NOTE: No data available for 1954-1955 Baltimore Bullets

## Import additional historical game by game data from basketball-reference

Starting from the 1983-1984 season, basketball-reference.com has game by game advanced statistics such as ORtg, DRtg, and eFG%

[Example boxscore](https://www.basketball-reference.com/boxscores/201803050CHI.html)

In [ ]:
def boxscore_links_for_date(date):
    '''
    get list of basketball-reference links to boxscores for games on given date
    
    date: datetime.datetime object with year, month, and day specified
    
    returns list of urls to basketball-reference single game boxscores for given date
    '''
    link = "https://www.basketball-reference.com/boxscores/?month={}&day={}&year={}".format(date.month, date.day, date.year)
    result = requests.get(link)
    soup = BeautifulSoup(result.content, "html.parser")
    return ["http://www.basketball-reference.com" + game.find("a").get("href") for game in soup.find_all("td", {"class": "right gamelink"})]

In [ ]:
def boxscore_dict_for_link(link):
    '''
    advanced box score stats as dictionary from basketball-reference boxscore link
    
    link: string, link to single game basketball-reference boxscore
    
    returns dictionary with advanced stats for home (team1) and away (team2) teams for boxscore linked
    NOTE: basketball-reference only supports single game advanced stats starting from 1983-1984 NBA season
    '''
    result = requests.get(link)
    soup = BeautifulSoup(result.content, "html.parser")
    from bs4 import Comment
    assert soup.find("div", id="all_four_factors") != None, "Advanced box score metrics only available for dates with at least 1 NBA game starting from 1983-1984 season"
    for c in soup.find("div", id="all_four_factors").children:
        if type(c) == Comment:
            s_ind = c.index("<table")
            e_ind = c.index("</table>")
            table_html = c[s_ind:e_ind+8]
            break
    df = pd.read_html(table_html, header=1, index_col=0)[0]
    df["DRtg"] = df["ORtg"].values[::-1]
    df["NetRtg"] = df["ORtg"] - df["DRtg"]
    d = {}
    for i, team_name in enumerate(df.index):
        prefix = "team2_" if i == 0 else "team1_"
        for col in df.columns:
            d[prefix+col] = df.loc[team_name, col]
    return d

### Option 1) Download data

#### Crawling through FiveThirtyEight nba_elo data to find advanced box score metrics for all games since 1983-1984 NBA season

In [ ]:
start_year = 2017
end_year = 2018

session = subprocess.Popen(['python', 'scrape_boxscores.py', start_year, end_year], stdout=PIPE, stderr=PIPE)
stdout, stderr = session.communicate()

print(stdout)
if stderr:
    raise Exception("Error "+str(stderr))

#### Concatenate outputs from script to one csv

In [ ]:
df_boxscores_1984_2018 = pd.concat([pd.read_csv(fn, index_col=0, parse_dates=[1], infer_datetime_format=True) for fn in sorted(glob.glob("../Data/nba_game_data*.csv"))])
df_boxscores_1984_2018.to_csv("../Data/nba_boxscores_1984_2018.csv")

### Option 2) Load offline data

In [4]:
df_boxscores_1984_2018 = pd.read_csv("../Data/nba_boxscores_1984_2018.csv", index_col=0, parse_dates=[1], infer_datetime_format=True)
df_boxscores_1984_2018.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carmelo1_pre,carmelo2_pre,carmelo1_post,carmelo2_post,carmelo_prob1,carmelo_prob2,score1,score2,team2_Pace,team2_eFG%,team2_TOV%,team2_ORB%,team2_FT/FGA,team2_ORtg,team2_DRtg,team2_NetRtg,team1_Pace,team1_eFG%,team1_TOV%,team1_ORB%,team1_FT/FGA,team1_ORtg,team1_DRtg,team1_NetRtg
24793,1983-10-28,1984,0,NaN,CLE,NYK,1381.7836,1585.6497,0.354824,0.645176,1376.2715,1591.1617,NaN,NaN,NaN,NaN,NaN,NaN,106.0,113.0,90.7,0.590,15.6,48.5,0.269,124.5,116.8,7.7,90.7,0.513,15.8,48.6,0.333,116.8,124.5,-7.7
24794,1983-10-28,1984,0,NaN,PHI,WSB,1680.4036,1542.6188,0.797186,0.202814,1682.3088,1540.7135,NaN,NaN,NaN,NaN,NaN,NaN,117.0,114.0,103.5,0.561,18.1,27.8,0.268,110.1,113.0,-2.9,103.5,0.512,16.6,36.6,0.402,113.0,110.1,2.9
24795,1983-10-28,1984,0,NaN,DEN,UTA,1538.7487,1430.0630,0.768754,0.231246,1543.8461,1424.9656,NaN,NaN,NaN,NaN,NaN,NaN,139.0,125.0,108.1,0.500,5.8,21.3,0.250,115.6,128.6,-13.0,108.1,0.539,8.5,41.3,0.284,128.6,115.6,13.0
24796,1983-10-28,1984,0,NaN,POR,SDC,1529.3440,1378.8689,0.808741,0.191259,1533.4425,1374.7704,NaN,NaN,NaN,NaN,NaN,NaN,107.0,93.0,95.0,0.500,17.8,20.6,0.348,97.9,112.6,-14.7,95.0,0.506,15.3,27.3,0.309,112.6,97.9,14.7
24797,1983-10-28,1984,0,NaN,SEA,GSW,1540.0374,1436.8497,0.763080,0.236920,1532.6714,1444.2157,NaN,NaN,NaN,NaN,NaN,NaN,109.0,110.0,101.6,0.472,15.3,47.7,0.292,108.3,107.3,1.0,101.6,0.534,19.1,23.5,0.405,107.3,108.3,-1.0


### Analyze Fields

Source: NBA game data (1984-2018) from [FiveThirtyEight Data Repo](https://github.com/fivethirtyeight/data) and [basketball-reference.com's boxscore data](https://www.basketball-reference.com/boxscores/)

See [FiveThirtyEight NBA Predictions](https://projects.fivethirtyeight.com/2018-nba-predictions/) for explanation of Elo and CARM-Elo

| Name           |  Type         | Description                                                                        
| -------------  |-------------  | ----------------------------------------------------------------------------------|
| date           | datetime      |   Game date                                                                       |                      
| season         | int           |   End year of season                                                              |
| neutral        | bool          |   1=neutral home court                                                            |
| playoff        | string        |   1946-2015: t = playoffs. Starting in 2016: q=conference quarterfinals, s=conference semifinals, c=conference final,f=finals                                                                 |
| team1          | string        |   Home team initials                                                              |
| team2          | string        |   Away team initials                                                              |
| elo1_pre       | double        |   Home team Elo before game                                                       |
| elo2_pre       | double        |   Away team Elo before game                                                       |
| elo_prob1      | double        |   Win probability based on elo1_pre                                               |
| elo_prob2      | double        |   Win probability based on elo2_pre                                               |
| elo1_post      | double        |   Home team Elo after game                                                        |
| elo2_post      | double        |   Away team Elo after game                                                        |
| carmelo1_pre\* | double        |   Home team CARM-Elo before game                                                  |
| carmelo2_pre\* | double        |   Away team CARM-Elo before game                                                  |
| carmelo1_post\*| double        |   Home team CARM-Elo after game                                                   |
| carmelo2_post\*| double        |   Away team CARM-Elo after game                                                   |
| carmelo_prob1\*| double        |   Win probability based on carmelo1_pre                                           |
| carmelo_prob2\*| double        |   Win probability based on carmelo2_pre                                           |
| score1         | int           |   Home team final score                                                           |
| score2         | int           |   Away team final score                                                           |
| team1_pace     | double        |   Home team possessions per 48 mins                                               |
| team1_eFG%     | double        |   Home team eFG%                                                                  |
| team1_TOV%     | double        |   Home team TOV%                                                                  |
| team1_ORB%     | double        |   Home team ORB%                                                                  |
| team1_FT/FGA   | double        |   Home team FT/FGA                                                                |
| team1_ORtg     | double        |   Home team ORtg                                                                  |
| team1_DRtg     | double        |   Home team DRtg                                                                  |
| team1_NetRtg   | double        |   Home team NetRtg                                                                |
| team2_pace     | double        |   Away team possessions per 48 mins                                               |
| team2_eFG%     | double        |   Away team eFG%                                                                  |
| team2_TOV%     | double        |   Away team TOV%                                                                  |
| team2_ORB%     | double        |   Away team ORB%                                                                  |
| team2_FT/FGA   | double        |   Away team FT/FGA                                                                |
| team2_ORtg     | double        |   Away team ORtg                                                                  |
| team2_DRtg     | double        |   Away team DRtg                                                                  |
| team2_NetRtg   | double        |   Away team NetRtg                                                                |


\* data only available from 2015-2018


# Consolidate datasets

1. Game by game ELO data from FiveThirtyEight starting from [1946 season](https://projects.fivethirtyeight.com/nba-model/nba_elo.csv)
2. basketball-reference season summaries w/ advanced stats available starting from [1951 season](https://www.basketball-reference.com/leagues/NBA_1951.html)
3. Game by game boxscores w/ advanced stats from basketball-reference starting from [1984 season](https://www.basketball-reference.com/boxscores/?month=10&day=28&year=1983)

**Current datasets** 
* nba_season_summaries_1951_2018.csv - 2
* nba_boxscores_1984_2018.csv - 1 + 3

**Desired datasets**
* Season summaries 1984-2018 with regular season ELO (ELO rating after last game of regular season)

## Outline

* Filter nba_season_summaries_1951_2018.csv for seasons between 1984 and 2018
* Find abbreviations for all team names from 1984-2018 (summaries use full name, boxscores use initials)
* Use nba_boxscores_1984_2018.csv to determine end of regular season ELO for every team from 1984 to 2018

** Filter nba_season_summaries_1951_2018.csv for seasons between 1984 and 2018 **

In [197]:
df_season_summaries_1984_2018 = df_season_summaries_1951_2018.loc[1984:2018].copy()
df_season_summaries_1984_2018.head()

Rk   Age     W     L    PW    PL   MOV   SOS  \
Season Team                                                                  
1984   Atlanta Hawks        14.0  26.6  40.0  42.0  37.0  45.0 -1.29  0.22   
       Boston Celtics        1.0  28.0  62.0  20.0  57.0  25.0  6.56 -0.14   
       Chicago Bulls        23.0  24.4  27.0  55.0  28.0  54.0 -5.18  0.50   
       Cleveland Cavaliers  21.0  25.4  28.0  54.0  30.0  52.0 -4.26  0.55   
       Dallas Mavericks     11.0  24.7  43.0  39.0  42.0  40.0  0.43 -0.27   

                             SRS   ORtg   DRtg  NetRtg  Pace    FTr   3PAr  \
Season Team                                                                  
1984   Atlanta Hawks       -1.08  105.5  106.9    -1.4  95.3  0.355  0.016   
       Boston Celtics       6.42  110.9  104.4     6.5  99.7  0.333  0.032   
       Chicago Bulls       -4.69  102.4  107.5    -5.1  99.8  0.360  0.017   
       Cleveland Cavaliers -3.71  104.1  108.5    -4.4  97.4  0.301  0.023   
       Dallas Mavericks     0.15  110.0  109.6     0.4  99.0  0.325  0.025   

                              TS%   eFG%  TOV%  ORB%  FT/FGA  eFG%.1  TOV%.1  \
Season Team                                                                    
1984   Atlanta Hawks        0.529  0.476  14.4  31.6   0.270   0.482    15.1   
       Boston Celtics       0.554  0.504  14.6  34.2   0.264   0.475    13.8   
       Chicago Bulls        0.526  0.475  16.3  32.3   0.268   0.497    15.6   
       Cleveland Cavaliers  0.512  0.468  14.0  33.5   0.224   0.490    13.2   
       Dallas Mavericks     0.547  0.503  13.6  31.7   0.245   0.503    14.4   

                            DRB%  FT/FGA.1                  Arena   Attend.  \
Season Team                                                                   
1984   Atlanta Hawks        65.2     0.268          Omni Coliseum  286049.0   
       Boston Celtics       69.7     0.225          Boston Garden  606857.0   
       Chicago Bulls        67.2     0.266        Chicago Stadium  256430.0   
       Cleveland Cavaliers  70.8     0.280  Coliseum at Richfield  208094.0   
       Dallas Mavericks     65.7     0.232          Reunion Arena  538162.0   

                            Attend./G  
Season Team                            
1984   Atlanta Hawks              NaN  
       Boston Celtics             NaN  
       Chicago Bulls              NaN  
       Cleveland Cavaliers        NaN  
       Dallas Mavericks           NaN

**$\checkmark$ Filter nba_season_summaries_1951_2018.csv for seasons between 1984 and 2018**

** Find abbreviations for all team names from 1984-2018 **

In [198]:
teams = set([index[1] for index, _ in df_season_summaries_1984_2018.iterrows() if index[1] != "League Average"])
teams

{'Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Bobcats',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'Kansas City Kings',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Jersey Nets',
 'New Orleans Hornets',
 'New Orleans Pelicans',
 'New Orleans/Oklahoma City Hornets',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'San Diego Clippers',
 'Seattle SuperSonics',
 'Toronto Raptors',
 'Utah Jazz',
 'Vancouver Grizzlies',
 'Washington Bullets',
 'Washington Wizards'}

In [199]:
nba_1984_2018_initials = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BRK',
    'Charlotte Hornets': 'CHH',
    'Charlotte Bobcats': 'CHO',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Kansas City Kings': 'KCK',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Hornets' : 'NOP',
    'New Orleans/Oklahoma City Hornets': 'NOK',
    'New Orleans Pelicans': 'NOP',
    'New Jersey Nets': 'NJN',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Diego Clippers': 'SDC',
    'San Antonio Spurs': 'SAS',
    'Seattle SuperSonics': 'SEA',
    'Toronto Raptors': 'TOR',
    'Vancouver Grizzlies': 'VAN',
    'Utah Jazz': 'UTA',
    'Washington Bullets': 'WSB',
    'Washington Wizards': 'WAS'
}

In [200]:
assert len(teams) == len(nba_1984_2018_initials.values())
for initial in nba_1984_2018_initials.values():
    assert df_boxscores_1984_2018[df_boxscores_1984_2018["team1"] == initial].shape[0] != 0, "Non-existing initial {}".format(initial)

** $\checkmark$ Find abbreviations for all team names from 1984-2018 **



** Use nba_boxscores_1984_2018.csv to determine end of regular season ELO for every team from 1984 to 2018 **

In [201]:
def regular_season_elo(boxscore_df, team_initials, season):
    '''
    Find the end of regular season ELO rating for a team (or ELO after latest game played to date, if season ongoing)
    
    boxscore_df: DataFrame, see https://projects.fivethirtyeight.com/nba-model/nba_elo.csv for format
    team_initials: string, official three letter initial for team
    season: int, year of season (ex: 2017 refers to 2016-17 season)
    '''
    # handle edge case where 'new' Charlotte Hornets use initials CHO in basketball-reference and ELO dataset 
    if team_initials == "CHH" and season in list(range(2015, datetime.datetime.now().year + 1)):
        team_initials = "CHO"
    team_mask = (boxscore_df["team1"] == team_initials) | (boxscore_df["team2"] == team_initials)
    season_mask = boxscore_df["season"] == season
    regular_season_mask = pd.isnull(boxscore_df["playoff"])
    last_game_indx = boxscore_df[team_mask & season_mask & regular_season_mask].tail(1).index[0]
    team1 = boxscore_df.loc[last_game_indx]["team1"]
    return boxscore_df.loc[last_game_indx]["elo1_post"] if team1 == team_initials else boxscore_df.loc[last_game_indx]["elo2_post"]

In [202]:
seasons_and_elos = [(index[0], regular_season_elo(df_boxscores_1984_2018, nba_1984_2018_initials[index[1]], index[0]) if index[1] != "League Average" else -1) for index, row in df_season_summaries_1984_2018.iterrows()]

In [203]:
season_elos = []
for indx, (season, elo_rating) in enumerate(seasons_and_elos):
    if elo_rating == -1:
        # compute average for year
        season_elos.append(np.mean([r for s, r in seasons_and_elos if s == season and r != -1]))
    else:
        season_elos.append(elo_rating)
df_season_summaries_1984_2018["ELO"] = season_elos

In [226]:
df_season_summaries_1984_2018

Rk   Age     W     L    PW    PL   MOV   SOS  \
Season Team                                                                     
1984   Atlanta Hawks           14.0  26.6  40.0  42.0  37.0  45.0 -1.29  0.22   
       Boston Celtics           1.0  28.0  62.0  20.0  57.0  25.0  6.56 -0.14   
       Chicago Bulls           23.0  24.4  27.0  55.0  28.0  54.0 -5.18  0.50   
       Cleveland Cavaliers     21.0  25.4  28.0  54.0  30.0  52.0 -4.26  0.55   
       Dallas Mavericks        11.0  24.7  43.0  39.0  42.0  40.0  0.43 -0.27   
       Denver Nuggets          15.0  27.0  38.0  44.0  38.0  44.0 -1.10 -0.17   
       Detroit Pistons          4.0  25.4  49.0  33.0  50.0  32.0  3.59 -0.06   
       Golden State Warriors   20.0  25.6  37.0  45.0  32.0  50.0 -3.40  0.05   
       Houston Rockets         18.0  26.8  29.0  53.0  33.0  49.0 -3.09 -0.04   
       Indiana Pacers          22.0  24.4  26.0  56.0  28.0  54.0 -4.82  0.57   
       Kansas City Kings       16.0  26.2  38.0  44.0  37.0  45.0 -1.48 -0.15   
       League Average           NaN  26.4   NaN   NaN  41.0  41.0  0.00  0.00   
       Los Angeles Lakers       5.0  27.5  54.0  28.0  50.0  32.0  3.76 -0.44   
       Milwaukee Bucks          2.0  28.4  50.0  32.0  52.0  30.0  4.16 -0.12   
       New Jersey Nets          8.0  25.5  45.0  37.0  44.0  38.0  1.10  0.17   
       New York Knicks          3.0  27.2  47.0  35.0  51.0  31.0  3.84 -0.05   
       Philadelphia 76ers       7.0  27.7  52.0  30.0  47.0  35.0  2.20  0.20   
       Phoenix Suns            10.0  27.2  41.0  41.0  43.0  39.0  0.89 -0.24   
       Portland Trail Blazers   6.0  25.8  48.0  34.0  50.0  32.0  3.55 -0.42   
       San Antonio Spurs       13.0  27.4  37.0  45.0  40.0  42.0 -0.27 -0.23   
       San Diego Clippers      19.0  25.5  30.0  52.0  33.0  49.0 -3.26  0.04   
       Seattle SuperSonics     12.0  27.1  42.0  40.0  41.0  41.0 -0.17 -0.17   
       Utah Jazz                9.0  26.8  45.0  37.0  44.0  38.0  1.13 -0.32   
       Washington Bullets      17.0  26.2  35.0  47.0  33.0  49.0 -2.89  0.53   
1985   Atlanta Hawks           15.0  25.5  34.0  48.0  37.0  45.0 -1.45  0.31   
       Boston Celtics           3.0  28.5  63.0  19.0  57.0  25.0  6.65 -0.18   
       Chicago Bulls           14.0  25.2  38.0  44.0  39.0  43.0 -0.84  0.34   
       Cleveland Cavaliers     16.0  26.6  36.0  46.0  34.0  48.0 -2.76  0.48   
       Dallas Mavericks         8.0  25.0  44.0  38.0  47.0  35.0  2.17 -0.37   
       Denver Nuggets           7.0  28.3  52.0  30.0  47.0  35.0  2.44 -0.39   
...                             ...   ...   ...   ...   ...   ...   ...   ...   
2018   Boston Celtics           4.0  24.6  45.0  20.0  42.0  23.0  4.31 -0.35   
       Brooklyn Nets           24.0  25.2  20.0  45.0  24.0  41.0 -4.25  0.06   
       Charlotte Hornets       17.0  26.7  28.0  37.0  32.0  33.0 -0.46  0.36   
       Chicago Bulls           28.0  24.5  22.0  42.0  20.0  44.0 -6.03  0.41   
       Cleveland Cavaliers     16.0  30.8  38.0  26.0  33.0  31.0  0.38 -0.18   
       Dallas Mavericks        22.0  28.1  20.0  45.0  27.0  38.0 -2.71  0.55   
       Denver Nuggets          12.0  24.8  35.0  30.0  34.0  31.0  0.85  0.16   
       Detroit Pistons         20.0  26.0  29.0  36.0  30.0  35.0 -1.26  0.25   
       Golden State Warriors    1.0  29.0  50.0  14.0  48.0  16.0  8.58  0.03   
       Houston Rockets          2.0  29.7  51.0  13.0  48.0  16.0  8.88 -0.26   
       Indiana Pacers          15.0  26.1  37.0  28.0  35.0  30.0  1.25 -0.66   
       League Average           NaN  26.6   NaN   NaN  32.0  32.0  0.00  0.00   
       Los Angeles Clippers    13.0  27.1  34.0  29.0  34.0  29.0  1.03 -0.05   
       Los Angeles Lakers      21.0  23.6  29.0  35.0  29.0  35.0 -1.63  0.00   
       Memphis Grizzlies       26.0  26.4  18.0  46.0  22.0  42.0 -4.69  0.46   
       Miami Heat              18.0  26.4  34.0  31.0  32.0  33.0 -0.26 -0.15   
       Milwaukee Bucks         19.0  25.7  34.0  31.0  31.0  34.

** $\checkmark$ Use nba_boxscores_1984_2018.csv to determine end of regular season ELO for every team from 1984 to 2018 **

In [ ]:
df_season_summaries_1984_2018.to_csv("../Data/nba_season_summaries_1984_2018.csv")

# $\checkmark$ Consolidate datasets



# Calculate moving SRS

In [88]:
mask_2018 = df_boxscores_1984_2018["season"] == 2018
mask_max_date = df_boxscores_1984_2018["date"] < datetime.datetime(2018, 3, 7)

df_2018_boxscores = df_boxscores_1984_2018[(mask_2018) & (mask_max_date)]
df_2018_boxscores.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carmelo1_pre,carmelo2_pre,carmelo1_post,carmelo2_post,carmelo_prob1,carmelo_prob2,score1,score2,team2_Pace,team2_eFG%,team2_TOV%,team2_ORB%,team2_FT/FGA,team2_ORtg,team2_DRtg,team2_NetRtg,team1_Pace,team1_eFG%,team1_TOV%,team1_ORB%,team1_FT/FGA,team1_ORtg,team1_DRtg,team1_NetRtg
65782,2017-10-17,2018,0,NaN,CLE,BOS,1647.989805,1532.470014,0.775674,0.224326,1650.129184,1530.330635,1648.0,1549.0,1650.308911,1546.691089,0.746290,0.253710,102.0,99.0,99.3,0.455,9.2,18.0,0.216,99.7,102.7,-3.0,99.3,0.488,15.3,19.6,0.253,102.7,99.7,3.0
65783,2017-10-17,2018,0,NaN,GSW,HOU,1760.609663,1574.467471,0.838508,0.161492,1751.819016,1583.258119,1761.0,1675.0,1753.884111,1682.115889,0.747495,0.252505,121.0,122.0,102.0,0.562,10.2,22.2,0.134,119.6,118.6,1.0,102.0,0.638,16.0,15.4,0.238,118.6,119.6,-1.0
65784,2017-10-18,2018,0,NaN,ORL,MIA,1390.229357,1552.809706,0.410901,0.589099,1400.663642,1542.375421,1458.0,1483.0,1464.397752,1476.602248,0.598634,0.401366,116.0,109.0,105.2,0.475,10.7,22.0,0.129,103.6,110.3,-6.7,105.2,0.522,12.0,25.0,0.244,110.3,103.6,6.7
65785,2017-10-18,2018,0,NaN,DET,CHO,1456.654984,1473.216401,0.617821,0.382179,1464.992663,1464.878722,1427.0,1542.0,1439.104231,1529.895769,0.476536,0.523464,102.0,90.0,98.5,0.459,16.5,7.3,0.315,91.4,103.6,-12.2,98.5,0.474,7.3,17.0,0.115,103.6,91.4,12.2
65786,2017-10-18,2018,0,NaN,IND,BRK,1502.884837,1405.034022,0.757481,0.242519,1506.960938,1400.957921,1406.0,1381.0,1411.729285,1375.270715,0.671978,0.328022,140.0,131.0,113.3,0.543,15.6,25.0,0.309,115.7,123.6,-7.9,113.3,0.564,10.8,30.4,0.245,123.6,115.7,7.9


In [89]:
nba_initials = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BRK',
    'Charlotte Hornets': 'CHO',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

In [90]:
def margin_for_team(abbrev, margins):
    return sum(margins[abbrev]) / len(margins[abbrev])

In [91]:
def sos_for_team(abbrev, schedule, margins):
    opp_movs = []
    for abbrev, gp in schedule[abbrev].items():
        opp_movs += [margin_for_team(abbrev, margins)] * gp
    return sum(opp_movs) / len(opp_movs)

In [92]:
margins = {abbrev: [] for _, abbrev in nba_initials.items()}
schedule = {abbrev: {} for _, abbrev in nba_initials.items()}
for _, game in df_2018_boxscores.iterrows():
    home_team = game["team1"]
    home_score = game["score1"]
    away_team = game["team2"]
    away_score = game["score2"]
    # update rolling schedule for both teams
    gp = schedule[home_team].get(away_team, None)
    if gp is None:
        schedule[home_team][away_team] = 1
    else:
        schedule[home_team][away_team] = gp + 1
    gp = schedule[away_team].get(home_team, None)
    if gp is None:
        schedule[away_team][home_team] = 1
    else:
        schedule[away_team][home_team] = gp + 1
    mov_home_team = home_score - away_score
    mov_away_team = -mov_home_team
    margins[home_team].append(mov_home_team)
    margins[away_team].append(mov_away_team)
print(margin_for_team("GSW", margins))
print(sos_for_team("GSW", schedule, margins))

8.578125
-0.021849781078296507


In [244]:
df_season_summaries_1984_2018.loc[(2018), "League Average"]

Rk               NaN
Age             26.6
W                NaN
L                NaN
PW                32
PL                32
MOV                0
SOS                0
SRS                0
ORtg           108.6
DRtg           108.6
NetRtg             0
Pace            97.3
FTr            0.255
3PAr           0.337
TS%            0.557
eFG%           0.521
TOV%            13.1
ORB%            22.4
FT/FGA         0.196
eFG%.1         0.521
TOV%.1          13.1
DRB%            77.6
FT/FGA.1       0.196
Arena            NaN
Attend.       576103
Attend./G      17818
ELO          1504.81
Name: (2018, League Average), dtype: object

In [114]:
# Get 2018 season summary, dropping League Average row
df_2018_summary = df_season_summaries_1984_2018.drop("League Average", level=1).loc[2018]
sos_lst, margin_lst = zip(*[(sos_for_team(nba_initials[name], schedule, margins), margin_for_team(nba_initials[name], margins)) for name in df_2018_summary.index])
df_2018_summary["my_MOV"] = list(map(lambda x: round(x,2), margin_lst))
df_2018_summary["my_SOS"] = list(map(lambda x: round(x,2), sos_lst))
df_2018_summary["my_SRS"] = df_2018_summary["my_MOV"] + df_2018_summary["my_SOS"]
df_2018_summary[['SRS','MOV', 'SOS', 'my_SRS', 'my_MOV', 'my_SOS']]

,SRS,MOV,SOS,my_SRS,my_MOV,my_SOS
Team,,,,,,
Golden State Warriors,8.61,8.58,0.03,8.56,8.58,-0.02
Houston Rockets,8.61,8.88,-0.26,8.52,8.84,-0.32
Toronto Raptors,8.02,8.64,-0.62,8.08,8.75,-0.67
Boston Celtics,3.96,4.31,-0.35,3.95,4.31,-0.36
Minnesota Timberwolves,2.64,2.73,-0.09,2.62,2.73,-0.11
Philadelphia 76ers,2.54,2.11,0.42,2.59,2.11,0.48
Oklahoma City Thunder,2.45,2.58,-0.12,2.39,2.58,-0.19
Utah Jazz,2.45,2.02,0.43,2.19,1.73,0.46
San Antonio Spurs,2.40,2.94,-0.54,2.32,2.94,-0.62


In [115]:
df_2018_summary[['SRS','MOV', 'SOS', 'my_SRS', 'my_MOV', 'my_SOS']].mean()

SRS       0.015333
MOV       0.015000
SOS       0.001667
my_SRS    0.013667
my_MOV    0.012667
my_SOS    0.001000
dtype: float64